# Deep Learning with Python
# 5.5 - Heatmaps of Class Activation

- A visualization technique which is useful for understanding which parts of a specific image led a convnet to its final classification.
- Especially relevant in the case of a classification mistake. 
- Can also be used to identify specific objects within an image. 

## Class Activation Maps (CAMs)
- Producing heatmaps of class activation over input images.
- A class activation map is a 2D grid of scores associated with respect to a specific output class that is computed for **every location in any input image**.
- It indicates how important each location (pixel) in the image is for the convnet to classify the image in a specific class.
- We're using Grad-CAM: a technique that
    - Takes the output feature map of a convolution layer for a given input.
    - Weighs every channel in that feature map by the gradient of the class with respect to the channel.
- Intuitively
    - We're computing the activation of an image with respect to a specific channel.
    - And then computing the activation of that channel with respect to a specific class.
    - The result of this cascaded series of operations ultimately results int he activation of an image with respect to a specific class.
- Really iffy mathematical notation for my own understanding
$$\frac{\delta[image activation]}{\delta[channel]} \times \frac{\delta[channel]}{\delta[class]} = \frac{\delta[image activation]}{\delta[class]}$$

## Demo - African Elephants and VGG16

###  Initializing Model

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# This time we're including the densely connected classifier
model = VGG16(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 64373286793e3c8b2b4e3219cbf3544b so we will re-download the data.
  8986624/553467096 [..............................] - ETA: 50:45

### Preprocessing Image

Need to preprocess the image by
- load the image
- resize to 244 x 244 pixels
- convert it to a `numpy` `float32` array
- apply VGG16's built-in preprocessing rules

In [5]:
# Preprocessing an input image for VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

In [6]:
# Defining the path of the Elephant's image
img_path = '/Users/saads/OneDrive/Desktop/DL-Python/chapter-5/cc_elephant.jpg'

In [ ]:
# Load the image as a Python object (Python Imaging Library of size (224, 224))
img = image.load_img(img_path, target_size=(224, 224))

In [ ]:
# Convert the image to an array of shape (224, 224, 3)
x = image.img_to_array(img)

In [ ]:
# Adds a dimension to transform the array into a batch of size (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

In [ ]:
# Preprocess the batch with VGG's built-in rules for channel-wise color normalization
x = preprocess_input(x)

### Pretrained Network Predictions
Run the pretrained network on the image and decode its prediction vector back to human-readable format.

In [ ]:
preds = model.predict(x)

In [ ]:
# Print the prediction accuracy and class name for the top three guesses
print('Predicted:', decode_predictions(preds, top=3)[0])

### Setting up Grad-CAM

In [ ]:
# African Elephant entry was found to be at index 386 using argmax 
african_elephant_output = model.output[:, 386]

In [ ]:
# Output feature map of the block5_conv3 layer - the last layer in the VGG16 model
last_conv_layer = model.get_layer('block5_conv3')

In [7]:
from tensorflow.keras import backend as K

In [ ]:
# Gradient of the African Elephant class with regard to the output feature map of block5_conv3
grads = K.gradients(african_elephant_output, 
                   last_conv_layer.output[6])

In [ ]:
# Vector of shape (512, ) where each entry is the mean intensity
# of the gradient over a specific feature-map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

In [ ]:
# Multiplieseach channel in the feature-map array by
# "how important this channel is" w.r.t the `elephant` class
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

In [ ]:
# Access the values of the quantities we just defined
# pooled_grads and the output feature map of the block5_conv3 
# given the sample image
iterate = K.function([model.input], 
                    [pooled_grads, last_conv_layer.output[0]])

In [ ]:
pooled_grads_value, conv_layer_output_value = iterate(x)

In [ ]:
# Channel-wise mean of the resulting feature map is the heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=1)

### Heatmap Postprocessing

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)